In [1]:
import os
import pandas as pd
import numpy as np

np.random.seed(seed=42)

### 데이터 전처리

In [2]:
data_dir = "../data"
# preprocessed_output_dir = "../data/train"
raw_ratebeer = pd.read_json(os.path.join(data_dir, "raw_ratebeer.json"))

In [4]:
raw_ratebeer.tail(1)

review_score               review_time  \
3316697           3.7 2000-07-19 13:26:50+00:00   

                                               review_text  appearance  aroma  \
3316697  .................................................         4.0    7.0   

         palate  taste  overall profile_name  \
3316697     4.0    7.0     15.0      kublai3   

                                   beer_name  beer_id  brewer_id  abv  \
3316697  Appalachian Jolly Scot Scottish Ale   3324.0      574.0  5.2   

                style                                          image_url  
3316697  Scottish Ale  https://res.cloudinary.com/ratebeer/image/uplo...

### 1. 결측치 제거 (label) reviewscore, beer_id NaN 제거

In [12]:
raw_ratebeer

review_score                         review_time  \
0                 3.9 1970-01-01 00:27:32.635611803+00:00   
1                 3.9 1970-01-01 00:27:31.691102673+00:00   
2                 4.1 1970-01-01 00:27:15.078944710+00:00   
3                 4.1 1970-01-01 00:27:10.441894537+00:00   
4                 4.0 1970-01-01 00:27:09.684443170+00:00   
...               ...                                 ...   
3244721           2.1           2002-06-19 18:22:54+00:00   
3244722           2.4           2002-01-01 11:42:59+00:00   
3244723           2.0           2001-10-09 00:08:24+00:00   
3244724           2.8           2001-09-12 09:13:12+00:00   
3244725           1.6           2001-08-11 14:31:27+00:00   

                                               review_text  appearance  aroma  \
0        Tap at Fermentoren CPH. Poured a hazy golden c...         3.0    8.0   
1        Bottle I shared with JB.  Beautiful cloudy str...         4.0    8.0   
2        2019 vintage\nAroma is vinegar, sour apples, f...         4.0    8.0   
3        Golden with a white head - Malt aroma with som...         3.0    8.0   
4        Bouteille.\nRobe orangée, trouble, peu scintil...         4.0    8.0   
...                                                    ...         ...    ...   
3244721  A one-dimensional, predictable macrobrew.  Sha...         2.0    5.0   
3244722  Very hoppy and dry without alot of complexity....         3.0    2.0   
3244723  quite hoppy yet still tasted too much of prese...         2.0    4.0   
3244724  Little sweet, nice hops, bit too much bubbles....         4.0    7.0   
3244725  re-rated 3-2018; from supermarket Aldi - trans...         3.0    2.0   

         palate  taste  overall     profile_name              beer_name  \
0           4.0    8.0     16.0        jmgreenuk  Hill Farmstead Arthur   
1           4.0    8.0     15.0              Ron  Hill Farmstead Arthur   
2           4.0    9.0     16.0          olberic  Hill Farmstead Arthur   
3           4.0    9.0     17.0       bergstaden  Hill Farmstead Arthur   
4           4.0    8.0     16.0           ZayZay  Hill Farmstead Arthur   
...         ...    ...      ...              ...                    ...   
3244721     2.0    5.0      7.0     austinpowers         Ambar Especial   
3244722     2.0    5.0     12.0  DavefromChicago         Ambar Especial   
3244723     2.0    4.0      8.0        motelpogo         Ambar Especial   
3244724     3.0    7.0      7.0            jbrus         Ambar Especial   
3244725     2.0    3.0      6.0              Bov         Ambar Especial   

          beer_id  brewer_id  abv                                 style  \
0        131594.0    11233.0  6.0         Saison / Farmhouse / Grisette   
1        131594.0    11233.0  6.0         Saison / Farmhouse / Grisette   
2        131594.0    11233.0  6.0         Saison / Farmhouse / Grisette   
3        131594.0    11233.0  6.0         Saison / Farmhouse / Grisette   
4        131594.0    11233.0  6.0         Saison / Farmhouse / Grisette   
...           ...        ...  ...                                   ...   
3244721    8666.0     1474.0  5.2  Pale Lager - International / Premium   
3244722    8666.0     1474.0  5.2  Pale Lager - International / Premium   
3244723    8666.0     1474.0  5.2  Pale Lager - International / Premium   
3244724    8666.0     1474.0  5.2  Pale Lager - International / Premium   
3244725    8666.0     1474.0  5.2  Pale Lager - International / Premium   

                                                 image_url  
0        https://res.cloudinary.com/ratebeer/image/uplo...  
1        https://res.cloudinary.com/ratebeer/image/uplo...  
2        https://res.cloudinary.com/ratebeer/image/uplo...  
3        https://res.cloudinary.com/ratebeer/image/uplo...  
4        https://res.cloudinary.com/ratebeer/image/uplo...  
...                                                    ...  
3244721  https://res.cloudinary.com/ratebeer/image/uplo...  
3244722  http

In [56]:
# reviewscore
print(raw_ratebeer.review_score.isna().sum())

# 인덱스 초기화
raw_ratebeer = raw_ratebeer[~raw_ratebeer.review_score.isna()]
raw_ratebeer.index = range(len(raw_ratebeer))

28


In [57]:
# beer_id
print(raw_ratebeer.beer_id.isna().sum())

# 인덱스 초기화
raw_ratebeer = raw_ratebeer[~raw_ratebeer.beer_id.isna()]
raw_ratebeer.index = range(len(raw_ratebeer))

779


### 2. user_id 등록

In [58]:
n_user = raw_ratebeer.profile_name.nunique()
print(f"n_user= {n_user:,}")

n_user= 60,586


In [59]:
profile_name2idx = {profile_name:user_id for user_id, profile_name in enumerate(raw_ratebeer.profile_name.unique())}
raw_ratebeer["user_id"] = [profile_name2idx[profile_name] for profile_name in raw_ratebeer.profile_name]

raw_ratebeer.head(1)

review_score                       review_time  \
0           3.9  2022-05-15 17:26:51.803000+00:00   

                                         review_text  appearance  aroma  \
0  Tap at Fermentoren CPH. Poured a hazy golden c...         3.0    8.0   

   palate  taste  overall profile_name              beer_name   beer_id  \
0     4.0    8.0     16.0    jmgreenuk  Hill Farmstead Arthur  131594.0   

   brewer_id  abv                          style  \
0    11233.0  6.0  Saison / Farmhouse / Grisette   

                                           image_url  user_id  
0  https://res.cloudinary.com/ratebeer/image/uplo...        0

### 3. 타입 변경

In [60]:
raw_ratebeer["beer_id"] = raw_ratebeer["beer_id"].astype(int)
raw_ratebeer["brewer_id"] = raw_ratebeer["brewer_id"].astype(int)
raw_ratebeer["user_id"] = raw_ratebeer["user_id"].astype(int)

### 4. Text 전처리

In [61]:
special_class1 = ["\\", "\cX", "\d", "\D", "\f", "\n", "\r", "\s", "\S", "\t", "\v", "\w", "\W", "\0", "[\b]"] # "\xhh", "\uhhhh", "\uhhhhh"
special_class2 = ["'", '"']

In [62]:
temp_text = raw_ratebeer["review_text"]


In [63]:
for special_str in special_class1:
    temp_text = temp_text.apply(lambda x: x.replace(special_str," "))
for special_str in special_class2:
    temp_text = temp_text.apply(lambda x: x.replace(special_str,""))

In [64]:
# meaningless (......)
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))
temp_text = temp_text.apply(lambda x: x.replace(".....", ".."))
temp_text = temp_text.apply(lambda x: x.replace("----", "-"))

# special (delimeter)
temp_text = temp_text.apply(lambda x: x.replace("|"," "))

In [65]:
# double check
for i in special_class1:
    counts = 0  
    for texts in temp_text:
        if i in texts:
            counts +=1
        # print(texts)
        # break
    print(i.encode("unicode_escape"), counts, end=" | ") # fr"{i}", ":", 

for i in special_class2:
    counts = 0  
    for texts in temp_text:
        if i in texts:
            counts +=1
        # print(texts)
        # break
    print(i.encode("unicode_escape"), counts, end=" | ") # fr"{i}", ":", 

counts = 0  
for texts in temp_text:
    if "|" in texts:
        counts +=1
    # print(texts)
    # break
print(r"|", counts)


# 적용
raw_ratebeer["review_text"] = temp_text


b'\\\\' 0 | b'\\\\cX' 0 | b'\\\\d' 0 | b'\\\\D' 0 | b'\\x0c' 0 | b'\\n' 0 | b'\\r' 0 | b'\\\\s' 0 | b'\\\\S' 0 | b'\\t' 0 | b'\\x0b' 0 | b'\\\\w' 0 | b'\\\\W' 0 | b'\\x00' 0 | b'[\\x08]' 0 | b"'" 0 | b'"' 0 | | 0


In [66]:
## 저장
raw_ratebeer.to_csv(os.path.join(data_dir,"preprocessed_ratebeer.tsv"), sep = "\t")

## DB에 저장할 것

In [67]:
# target data

In [68]:
target_beer_id_list = [ 8666,  75661, 614833, 169964,    251,  10726,   5588,   5571,
                        703,   6668,   7656,   2228, 135361,    473,  36249,    709,
                        742,   4007,  72025,  58275, 221535, 291276,
                        2469,   2468,   1088,    721,  30913,    814,  75071,
                        715,   1267,     55,  17334,     37, 493355,  64253,  34662,
                        506847,  73158,    399,   5159,   3101, 315443,    268,    390,
                        71469,  35424,   4701,   2514,   2516, 288954,    405,
                        136785,    647, 632627,   3029,    717, 553245, 580686,
                        21936, 667837,   2811,    729,  71458,  86387,   1478,
                        146878, 146725, 149060,  15747,   3126, 480225, 268685,
                        730, 211962,  64518, 621308,   7580, 452525,   1417,   1486, 221]

In [69]:
df_target_beer_id = pd.DataFrame()
df_target_beer_id["beer_id"] = target_beer_id_list
df_target_beer_id.tail(3)

beer_id
79     1417
80     1486
81      221

In [70]:
df_target_beer_id.to_csv(os.path.join(data_dir, "db_target_beer.csv"), index = False)

In [71]:
from datetime import datetime

#### DB-name : user ####
# user
user = raw_ratebeer.groupby('user_id')[['user_id', 'profile_name']].head(1).copy()
n_user = len(user)
user["password"] = [str(num).zfill(8) for num in np.random.randint(low = 0, high = 99999999, size = n_user)]
user["gender"] = "-"
user["birth"] = [datetime.strptime("1000-01-01 00:00:00.001", "%Y-%m-%d %H:%M:%S.%f")]*n_user
user = user[["user_id", "password", "profile_name", "gender", "birth"]]

print("n_user :", n_user)
user.head()

n_user : 60586


user_id  password profile_name gender                       birth
0        0  09358763    jmgreenuk      -  1000-01-01 00:00:00.001000
1        1  77442728          Ron      -  1000-01-01 00:00:00.001000
2        2  00732718      olberic      -  1000-01-01 00:00:00.001000
3        3  47592388   bergstaden      -  1000-01-01 00:00:00.001000
4        4  17853006       ZayZay      -  1000-01-01 00:00:00.001000

In [72]:
#### DB-name : beer ####
beer = raw_ratebeer.groupby("beer_name")[["beer_id","beer_name", "brewer_id", "abv", "style", "image_url"]].head(1).copy()
beer.index = range(len(beer))
print("n_beer :",  beer.shape[0])
beer.head()



n_beer : 9337


beer_id                        beer_name  brewer_id  abv  \
0   131594            Hill Farmstead Arthur      11233  6.0   
1   121536            O'Fallon Hemp Hop Rye       2399  5.5   
2    12617                Hartwall Urho III        814  4.7   
3    14940  Mendocino White Hawk Select IPA         65  7.0   
4    25933  Moorhouse's Blonde Witch (Cask)        400  4.4   

                           style  \
0  Saison / Farmhouse / Grisette   
1        Specialty Grain - Other   
2          Pale Lager - American   
3                  IPA - English   
4        Blonde Ale / Golden Ale   

                                           image_url  
0  https://res.cloudinary.com/ratebeer/image/uplo...  
1  https://res.cloudinary.com/ratebeer/image/uplo...  
2  https://res.cloudinary.com/ratebeer/image/uplo...  
3  https://res.cloudinary.com/ratebeer/image/uplo...  
4  https://res.cloudinary.com/ratebeer/image/uplo...

In [73]:
#### DB-name : review ####
review = raw_ratebeer[["user_id", "beer_id", "review_score", "review_text", "review_time", "appearance", "aroma", "palate", "taste", "overall"]].copy()
review.head()

user_id  beer_id  review_score  \
0        0   131594           3.9   
1        1   131594           3.9   
2        2   131594           4.1   
3        3   131594           4.1   
4        4   131594           4.0   

                                         review_text  \
0  Tap at Fermentoren CPH. Poured a hazy golden c...   
1  Bottle I shared with JB.  Beautiful cloudy str...   
2  2019 vintage Aroma is vinegar, sour apples, fr...   
3  Golden with a white head - Malt aroma with som...   
4  Bouteille. Robe orangée, trouble, peu scintill...   

                        review_time  appearance  aroma  palate  taste  overall  
0  2022-05-15 17:26:51.803000+00:00         3.0    8.0     4.0    8.0     16.0  
1  2022-05-04 19:05:02.673000+00:00         4.0    8.0     4.0    8.0     15.0  
2  2021-10-24 12:35:44.710000+00:00         4.0    8.0     4.0    9.0     16.0  
3  2021-08-31 20:31:34.537000+00:00         3.0    8.0     4.0    9.0     17.0  
4  2021-08-23 02:07:23.170000+00:00         4.0    8.0     4.0    8.0     16.0

In [74]:
for i in range(129700, 129804):
    print(f"--{i}------------------------------------------------------------------------------------")
    print(review.iloc[i]["review_text"])

--129700------------------------------------------------------------------------------------
From a bottle. Pours a thick black with a lasting tan head, roasted coffee aroma, well carbonated, thick and rich with roasted malt and bitter chocolate/coffee flavours. I like it a lot, a real lot.
--129701------------------------------------------------------------------------------------
Couleur noire, très légère mousse qui disparait. Fort nez de malt grillé, café et chocolat noir. Gout doux de malt grillé, café et alcool. Corps assez aqueux
--129702------------------------------------------------------------------------------------
Bottle. Color brown with small white head. Aroma malt, rozsted, chocolate, coffee. Taste sweet, roasted, chocolate, alcohol, bourbon.
--129703------------------------------------------------------------------------------------
Draft at Tap 52. Brown pour with a kaki head. Light toasted malt aromas. Light toasted malts and mild black coffee notes. Average body an

In [75]:
user.to_csv(os.path.join(data_dir, "db_files/db_user.tsv"), index = False, sep = "\t")
beer.to_csv(os.path.join(data_dir, "db_files/db_beer.tsv"), index = False, sep = "\t")
review.to_csv(os.path.join(data_dir, "db_files/db_review.tsv"), index = False, sep ="\t")

In [76]:
## 학습용 데이터 만들기

### 4. 리뷰 개수가 10개 미만인 유저 제거(이거 보류)

In [77]:
all_ratebeer = raw_ratebeer.copy()

In [78]:
# # 리뷰 개수 10개 이상인 유저 리스트
# over10user_list = (train_ratebeer["user_id"].value_counts() >= 10).where(lambda x: x == True).dropna().index.tolist()

# # 리뷰 개수 10개 이상인 유저 리스트를 받아와서 해당 유저들로만 구성된 데이터셋으로 업데이트
# train_ratebeer = train_ratebeer[train_ratebeer["user_id"].isin(over10user_list)]

# # 인덱스 초기화
# train_ratebeer.index = range(len(train_ratebeer))

### 8. Train Test split

In [79]:
n_user = all_ratebeer.user_id.nunique()
test_ratio = 0.1
n_test_user = int(n_user*test_ratio)

np.random.seed(42)
test_user_idx_list = np.random.choice(range(n_user), size = n_test_user, replace=False)
test_user_list = all_ratebeer.user_id.unique()[test_user_idx_list]

train_ratebeer = all_ratebeer[~all_ratebeer["user_id"].isin(test_user_list)].drop("review_text", axis=1)
test_ratebeer = all_ratebeer[all_ratebeer["user_id"].isin(test_user_list)].drop("review_text", axis=1)

print(len(train_ratebeer)) # 2929083 
print(len(test_ratebeer)) # 314552

2931605
312314


In [80]:
# 저장
train_ratebeer.to_csv("../data/standard/train_ratebeer_without_text.csv", index = False)
test_ratebeer.to_csv("../data/standard/test_ratebeer_without_text.csv", index = False)

In [81]:
########################################################################
########################################################################
########################################################################
########################################################################

## 데이터 셋 분리 [Testset 분리]
- 총 15000명 가량의 유저를 통해서 
- 한 2000명에 대해서 각 1

In [103]:
df_test = test_ratebeer.copy()


In [44]:
df_test = pd.read_csv("/opt/ml/workspace/final-project-level3-recsys-10/data/standard/test_ratebeer_without_text.csv")
target_beer_id_list = pd.read_csv("/opt/ml/workspace/final-project-level3-recsys-10/data/db_files/db_target_beer.csv")
target_beer_id_list = list(target_beer_id_list["beer_id"].astype(int))

print(f"n_rows : {len(df_test):,}")
print(f"n_user : {df_test.user_id.nunique()}")
print(f"n_item : {df_test.beer_id.nunique()}")

n_rows : 312,314
n_user : 6058
n_item : 9323


In [45]:
# 한 개를 맞출 수는 없다. 따라서 하나만 리뷰한 사람은 제외한다. (적어도 두 명의 유저가 있어야 하므로)

# 실제 우리의 케이스에 맞춰서, 4개의 아이템에 대해서 RMSE를 구한다.
# 그 중에서도 target 맥주를 잘 예측해야하므로, 해당 맥주를 중점으로 예측하도록한다. 
# 그래서 리뷰 수가 4개 이하인 유저들은 예측하지 않는다. 
over1user_in_Test = (df_test.value_counts("user_id") <= 4).where(lambda x: x == False).dropna().index.tolist()
df_test = df_test[df_test["user_id"].isin(over1user_in_Test)]
df_test.index = range(len(df_test))

print(f"n_rows : {len(df_test):,}")
print(f"n_user : {df_test.user_id.nunique()}")
print(f"n_item : {df_test.beer_id.nunique()}")

n_rows : 305,856
n_user : 2104
n_item : 9323


In [69]:
dict_test_input = dict()
dict_test_answer = dict()
input_len = 0

np.random.seed(42)
for _, (user_id, beer_id_list) in df_test.groupby("user_id")["beer_id"].apply(list).reset_index().iterrows():
    target_beers =  list(set(beer_id_list).intersection(target_beer_id_list))
    other_beers = list(set(beer_id_list)-set(target_beer_id_list))
    
    # 타겟 비어가 없는 데이터는 제외한다.
    if target_beers == []:
        continue

    # 4개보다 많을 경우. -> 랜덤 추출
    if len(target_beers) > 4:
        temp_output = np.random.choice(target_beers, size=4, replace=False)

    # 4개보다 적을 경우. -> 다 꺼내기
    else:
        temp_output = target_beers
        n_need_to_add = 4 - len(temp_output) # 4개보다 부족한 경우 채워야한다. 
        temp_output.extend( np.random.choice(other_beers, size=n_need_to_add, replace=False) )

    # input은 정답지로 구성한 output에 넣은 맥주 이외의 모든 맥주로 구성한다. | (option-1) input은 30개로 한정한다.
    input_item_list = [beer_id for beer_id in beer_id_list if beer_id not in temp_output][-30:]
    output_item_list = temp_output
    input_len += len(input_item_list)

    dict_test_input[user_id] = input_item_list
    dict_test_answer[user_id] = output_item_list

print(len(dict_test_input))
print(len(dict_test_answer))

1304
1304


In [70]:
df_test_input = pd.concat({k: pd.Series(v) for k, v in dict_test_input.items()})
df_test_answer = pd.concat({k: pd.Series(v) for k, v in dict_test_answer.items()})

In [71]:
df_test_input = df_test_input.reset_index()
df_test_input.columns = ["user_id", "order", "beer_id"]
df_test_input = df_test_input.drop("order", axis=1)
df_test_input = df_test_input.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_input))
df_test_input.head(3)

28297


user_id  beer_id  review_score                       review_time  \
0        4   145903           4.2  2021-08-19 22:01:20.193000+00:00   
1        4   114499           3.9  2021-01-25 03:00:13.110000+00:00   
2        4    12027           3.8  2021-08-23 02:21:37.730000+00:00   

   appearance  aroma  palate  taste  overall profile_name  \
0         4.0    9.0     4.0    8.0     17.0       ZayZay   
1         3.0    8.0     4.0    8.0     16.0       ZayZay   
2         4.0    8.0     4.0    7.0     15.0       ZayZay   

                           beer_name  brewer_id  abv  \
0        Hill Farmstead Double Citra      11233  8.0   
1  St-Ambroise Stout Impériale Russe        137  9.2   
2    Boon Oude Geuze Mariage Parfait        748  8.0   

                                        style  \
0  IIPA DIPA - Imperial / Double Hazy (NEIPA)   
1                            Stout - Imperial   
2                             Lambic - Gueuze   

                                           image_url  
0  https://res.cloudinary.com/ratebeer/image/uplo...  
1  https://res.cloudinary.com/ratebeer/image/uplo...  
2  https://res.cloudinary.com/ratebeer/image/uplo...

In [72]:
df_test_answer = df_test_answer.reset_index()
df_test_answer.columns = ["user_id", "order", "beer_id"]
df_test_answer = df_test_answer.drop("order", axis=1)
df_test_answer = df_test_answer.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_answer))
df_test_answer.head(3)

5216


user_id  beer_id  review_score                       review_time  \
0        4       37           1.9  2021-07-08 03:31:02.923000+00:00   
1        4      709           1.6  2021-01-11 01:47:51.440000+00:00   
2        4      473           1.7  2022-05-27 03:50:49.850000+00:00   

   appearance  aroma  palate  taste  overall profile_name          beer_name  \
0         3.0    3.0     2.0    3.0      8.0       ZayZay           Heineken   
1         2.0    3.0     1.0    3.0      7.0       ZayZay  Carlsberg Pilsner   
2         3.0    3.0     2.0    3.0      6.0       ZayZay          Budweiser   

   brewer_id  abv                                 style  \
0          9  5.0  Pale Lager - International / Premium   
1         43  4.6             Pilsener / Pils / Pilsner   
2         84  5.0                 Pale Lager - American   

                                           image_url  
0  https://res.cloudinary.com/ratebeer/image/uplo...  
1  https://res.cloudinary.com/ratebeer/image/uplo...  
2  https://res.cloudinary.com/ratebeer/image/uplo...

In [73]:
df_test_input.to_csv("../data/test/test_input_ratebeer_without_text_limit30.csv", index = False)
df_test_answer.to_csv("../data/test/test_output_ratebeer_without_text_limit30.csv", index = False)

: 

In [17]:
dict_test_input = dict()
dict_test_answer = dict()

np.random.seed(42)
for _, (user_id, beer_id_list) in df_test.groupby("user_id")["beer_id"].apply(list).reset_index().iterrows():
    # 기본적으로 target beer들 이외의 맥주는 제외하려고 한다.
    # 하지만, 유저의 맥주에 대한 전반적인 선호도 파악도 잘하고 있는지 확인해야하므로, 
    # 타겟 맥주를 제외하고, 10%+1개의 맥주만을 남겨두고 제외한다.

    target_beers =  list(set(beer_id_list).intersection(target_beer_id_list))
    n_target_beers = len(target_beers)

    other_beers = list(set(beer_id_list)-set(target_beer_id_list))
    n_other_beers = len(other_beers)

    if n_other_beers > 0 : 
        n_add = int(n_other_beers*0.1) + 1 # 10%+1개의 맥주만을 남겨두고
        random_choice_idx = np.random.choice(range(n_other_beers), size = n_add, replace=False).tolist()
        other_beers_to_input = np.array(other_beers)[random_choice_idx].tolist()
    else:
        other_beers_to_input = []
    
    n_mask = min(int(n_target_beers*0.5), 8)
    random_choice_idx = np.random.choice(range(n_target_beers), size = n_mask, replace=False).tolist()

    target_beers_to_answer = np.array(target_beers)[random_choice_idx].tolist()
    target_beers_to_input = list(set(target_beers)-set(target_beers_to_answer))

    input_list = other_beers_to_input + target_beers_to_input
    answer_list = target_beers_to_answer

    # print(input_list)
    # print(answer_list)

    if input_list == [] or answer_list == []:
        continue


    dict_test_input[user_id] = input_list
    dict_test_answer[user_id] = answer_list
    
    # 유저별로 최대 8개를 정답지로 구성한다.
    # 기본적으로 한국맥주 중의 50%를 정답지로 마스킹해준다.
    # min(len())
    # 최대 8개 

In [212]:
df_test_input = pd.concat({k: pd.Series(v) for k, v in dict_test_input.items()})
df_test_answer = pd.concat({k: pd.Series(v) for k, v in dict_test_answer.items()})
#코드참고 https://stackoverflow.com/questions/42869544/dictionary-of-lists-to-dataframe

In [213]:
df_test_input = df_test_input.reset_index()
df_test_input.columns = ["user_id", "order", "beer_id"]
df_test_input = df_test_input.drop("order", axis=1)
df_test_input = df_test_input.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_input))
df_test_input.head(3)


33282


user_id  beer_id  review_score                      review_time  \
0        4    12027           3.8 2021-08-23 02:21:37.730000+00:00   
1        4    59197           3.8 2020-10-24 01:01:04.880000+00:00   
2        4    59877           3.5 2021-04-10 00:07:47.060000+00:00   

   appearance  aroma  palate  taste  overall profile_name  \
0         4.0    8.0     4.0    7.0     15.0       ZayZay   
1         4.0    8.0     4.0    8.0     14.0       ZayZay   
2         3.0    8.0     3.0    7.0     14.0       ZayZay   

                         beer_name  brewer_id   abv             style  \
0  Boon Oude Geuze Mariage Parfait        748   8.0   Lambic - Gueuze   
1     Le Trou du Diable La Buteuse       6444  10.0            Tripel   
2   Dieu du Ciel! Rosée d'Hibiscus        364   5.9  Flavored - Other   

                                           image_url  
0  https://res.cloudinary.com/ratebeer/image/uplo...  
1  https://res.cloudinary.com/ratebeer/image/uplo...  
2  https://res.cloudinary.com/ratebeer/image/uplo...

In [214]:
df_test_answer = df_test_answer.reset_index()
df_test_answer.columns = ["user_id", "order", "beer_id"]
df_test_answer = df_test_answer.drop("order", axis=1)
df_test_answer = df_test_answer.merge(df_test, on=["user_id", "beer_id"])
print(len(df_test_answer))
df_test_answer.head(3)

3631


user_id  beer_id  review_score                      review_time  \
0        4       37           1.9 2021-07-08 03:31:02.923000+00:00   
1        4      473           1.7 2022-05-27 03:50:49.850000+00:00   
2        4      742           1.7 2020-10-24 21:25:51.827000+00:00   

   appearance  aroma  palate  taste  overall profile_name     beer_name  \
0         3.0    3.0     2.0    3.0      8.0       ZayZay      Heineken   
1         3.0    3.0     2.0    3.0      6.0       ZayZay     Budweiser   
2         2.0    3.0     2.0    3.0      7.0       ZayZay  Corona Extra   

   brewer_id  abv                                 style  \
0          9  5.0  Pale Lager - International / Premium   
1         84  5.0                 Pale Lager - American   
2        119  4.5                 Pale Lager - American   

                                           image_url  
0  https://res.cloudinary.com/ratebeer/image/uplo...  
1  https://res.cloudinary.com/ratebeer/image/uplo...  
2  https://res.cloudinary.com/ratebeer/image/uplo...

In [215]:
df_test_input.to_csv("../data/test/test_input_ratebeer_without_text.csv", index = False)
df_test_answer.to_csv("../data/test/test_output_ratebeer_without_text.csv", index = False)

In [ ]:
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################
########################################################################################################################################

### 8. training file making

In [12]:
raw_ratebeer[["user_id", "beer_id", "reviewscore", "reviewtime"]].isna().sum()

user_id        0
beer_id        0
reviewscore    0
reviewtime     0
dtype: int64

In [93]:
train_ratings = raw_ratebeer[["user_id", "beer_id", "reviewscore", "reviewtime"]]
train_ratings.columns = ["user", "item", "rating", "time"]
train_ratings.head(1)

user    item  rating                      time
0     0  131594     3.9  2022-05-15T17:26:51.803Z

In [96]:
train_ratings.to_csv(os.path.join(preprocessed_output_dir, "train_ratings.csv"), index=False)

### 8. attribute extraction (style)

In [97]:
style_df = raw_ratebeer.groupby("beer_id")["beer_id", "style"].head(1)
style_df.index = range(len(style_df))

<ipython-input-97-4ab092addaaa>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  style_df = raw_ratebeer.groupby("beer_id")["beer_id", "style"].head(1)


In [100]:
style_df

beer_id                                 style
0      131594         Saison / Farmhouse / Grisette
1      121536               Specialty Grain - Other
2       12617                 Pale Lager - American
3       14940                         IPA - English
4       25933               Blonde Ale / Golden Ale
...       ...                                   ...
9330   118583                       Red Ale - Irish
9331      786  Amber Lager - International / Vienna
9332    51898                      Stout - Imperial
9333     5067     Red Ale / International Amber Ale
9334     3324                          Scottish Ale

[9335 rows x 2 columns]

In [104]:
array, index = pd.factorize(style_df["style"])
style_df["style"] = array
style_df.groupby("beer_id")["style"].apply(list).to_json(os.path.join(preprocessed_output_dir, "rb_item2attributes.json"))